In [16]:
import pandas as pd
import numpy as np
from rmsle import ag_rmsle_clamped_scorer, ag_rmsle_scorer
from autogluon.tabular import TabularPredictor
from itertools import combinations

download = False

if download:
    !kaggle competitions download -c playground-series-s5e5 -p data/

    import zipfile, pathlib, os

    zip_path = pathlib.Path('data/playground-series-s5e5.zip')
    with zipfile.ZipFile(zip_path) as z:
        z.extractall(zip_path.parent)
    os.remove(zip_path)

def preprocess(train_df, test_df):
    train_df, test_df = handle_columns(train_df, test_df)

    for col in ['Age', 'Weight', 'Height', 'Duration', 'Heart_Rate', 'Body_Temp', 'BMI']:
        train_df, test_df = handle_outliers(train_df, test_df, col)

    train_df = remove_outliers(train_df, 'Calories')

    return train_df, test_df

def handle_columns(train_df, test_df):
    target = 'Calories'

    combined_df = pd.concat([train_df, test_df], ignore_index=True)

    combined_df['female'] = (combined_df['Sex'] == 'female').astype('int16')
    combined_df['male']   = (combined_df['Sex'] == 'male').astype('int16')
    combined_df = combined_df.drop(columns='Sex')
    combined_df['Age'] = combined_df['Age'].astype('float64')

    combined_df['BMI']  = combined_df['Weight'] / ((combined_df['Height'] / 100) ** 2)
    combined_df['BSA']  = 0.007184 * combined_df['Weight'] ** 0.425 * combined_df['Height'] ** 0.725
    combined_df['FFM']  = 0.407 * combined_df['Weight'] + 0.267 * combined_df['Height'] - 19.2
    combined_df['HRMax'] = 220.0 - combined_df['Age']
    combined_df['%HRMax'] = combined_df['Heart_Rate'] / combined_df['HRMax']
    combined_df['HRR']   = combined_df['HRMax'] - combined_df['Heart_Rate']
    combined_df['TRIMP'] = combined_df['Duration'] * combined_df['%HRMax']
    combined_df['Thermal_Load'] = (combined_df['Body_Temp'] - 37.0) * combined_df['Duration']
    combined_df['BMR'] = (
        10 * combined_df['Weight']
        + 6.25 * combined_df['Height']
        - 5 * combined_df['Age']
        + 5 * combined_df['male']
        - 161 * combined_df['female']
    )
    combined_df['Keytel'] = combined_df['Duration'] * (
        combined_df['Weight']     * (0.0475 * combined_df['male'] - 0.0302 * combined_df['female'])
      + combined_df['Heart_Rate'] * (0.151  * combined_df['male'] + 0.107  * combined_df['female'])
      - 13.17 * combined_df['male'] - 4.88 * combined_df['female']
    )

    num_cols  = combined_df.select_dtypes(['float64', 'int64']).drop(columns=[target]).columns
    new_cols  = {}

    """for col1 in num_cols:
        s1 = combined_df[col1]

        new_cols[f'{col1}^2']      = s1 ** 2
        new_cols[f'{col1}^3']      = s1 ** 3
        new_cols[f'log1p({col1})'] = np.log1p(s1)"""

    """for col1, col2 in combinations(num_cols, 2):
        s1, s2 = combined_df[col1], combined_df[col2]

        new_cols[f'{col1}+{col2}'] = s1 + s2
        new_cols[f'{col1}-{col2}'] = s1 - s2
        new_cols[f'{col1}*{col2}'] = s1 * s2

        if not (s2 == 0).any():
            new_cols[f'{col1}/{col2}'] = s1 / s2"""


    combined_df = pd.concat([combined_df, pd.DataFrame(new_cols, index=combined_df.index)], axis=1)

    for c in combined_df.select_dtypes('float64'):
        combined_df[c] = pd.to_numeric(combined_df[c], downcast='float')
    for c in combined_df.select_dtypes('int64'):
        combined_df[c] = pd.to_numeric(combined_df[c], downcast='integer')
    combined_df = combined_df.drop(columns=['male'])
    combined_df['female'] = combined_df['female'].astype('category')

    train_df = combined_df.iloc[:len(train_df)].copy().set_index('id')
    test_df  = combined_df.iloc[len(train_df):].copy().drop(columns=target).set_index('id')

    return train_df, test_df

def handle_outliers(train_df, test_df, column):
    q1 = train_df[column].quantile(0.25)
    q3 = train_df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    train_df = train_df[(train_df[column] >= lower_bound) & (train_df[column] <= upper_bound)]
    test_df[column] = test_df[column].clip(lower_bound, upper_bound)
    return train_df, test_df

def remove_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]


train, test = pd.read_csv('data/train.csv'), pd.read_csv('data/test.csv')
train, test = preprocess(train, test)

target = 'Calories/Duration'
train[target] = train['Calories'] / train['Duration']
train = train.drop(columns=['Calories'])

num_cols = [c for c in train.columns
            if train[c].dtype.kind in "fi" and c != target]
cat_cols = [c for c in train.columns
            if train[c].dtype.name == "category"]

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin

class CorrFilter(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.99, sample_size=20_000, random_state=0):
        self.threshold    = threshold
        self.sample_size  = sample_size
        self.random_state = random_state

    # ---------- FIT ---------------------------------------------------
    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        if self.sample_size and len(X) > self.sample_size:
            X = X.sample(self.sample_size, random_state=self.random_state)

        corr  = X.corr(numeric_only=True).abs()
        upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        self.to_drop_ = [c for c in upper.columns if any(upper[c] > self.threshold)]

        # remember original order
        self.cols_     = X.columns.to_numpy()
        self.drop_idx_ = [np.where(self.cols_ == c)[0][0] for c in self.to_drop_]
        return self

    # ---------- TRANSFORM ---------------------------------------------
    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            return X.drop(columns=self.to_drop_, errors="ignore")

        keep_mask = np.ones(X.shape[1], dtype=bool)
        keep_mask[self.drop_idx_] = False
        return X[:, keep_mask]

    # ---------- selector API ------------------------------------------
    def get_support(self, indices=False):
        mask = np.ones(len(self.cols_), dtype=bool)
        mask[self.drop_idx_] = False
        return np.where(mask)[0] if indices else mask

    # ---------- NEW: feature-name API ---------------------------------
    def get_feature_names_out(self, input_features=None):
        """
        Return the names of the columns that survive the correlation filter.
        """
        # scikit-learn will pass input_features when it knows them;
        # fall back to the order recorded in fit().
        if input_features is None:
            input_features = self.cols_
        input_features = np.asarray(input_features, dtype=object)

        mask = np.ones(len(input_features), dtype=bool)
        mask[self.drop_idx_] = False
        return input_features[mask]

In [22]:
import numpy as np
from sklearn.compose      import ColumnTransformer
from sklearn.preprocessing import (FunctionTransformer,
                                   StandardScaler,
                                   OneHotEncoder,
                                   PolynomialFeatures,
                                   QuantileTransformer)
from sklearn.pipeline     import Pipeline
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from sklearn.linear_model import ElasticNetCV
from sklearn.utils import resample

import time, gc, traceback, sys, os
from sklearn.base import clone

import numpy as np
import pandas as pd
import time
import sys
import traceback
from sklearn.base import clone

def stability_mask(pipe,
                           X, y,
                           n_loop     = 100,
                           frac       = 0.10,
                           cutoff     = 0.40,
                           seed       = 42,
                           echo_every = 10):
    """
    Bootstrap-based stability selection.

    Parameters
    ----------
    pipe        : sklearn Pipeline
        Must end with a selector exposing get_support().
    X, y        : pandas DataFrame / Series  (X MUST have column names)
    n_loop      : int   – number of bootstrap iterations
    frac        : float – fraction of rows per bootstrap sample (0–1)
    cutoff      : float – keep features selected in ≥ cutoff fraction
    seed        : int   – RNG seed
    echo_every  : int   – print heartbeat every N iterations

    Returns
    -------
    keep_mask : boolean array  (len = n_master_features)
        True for columns that passed the cutoff.
    freq      : float array
        Selection frequency for every master feature.
    """
    # ------------------------------------------------------------------
    # 1. Fit the *prep* part once to know the master column list
    #    (all steps before the final selector)
    # ------------------------------------------------------------------
    prep_only  = pipe[:-1]               # everything except last step
    prep_only  = clone(prep_only).fit(X)
    master_cols = prep_only.get_feature_names_out()
    n_master    = len(master_cols)

    # ------------------------------------------------------------------
    # 2. Prepare the vote counter
    # ------------------------------------------------------------------
    votes  = np.zeros(n_master, dtype=int)
    rng    = np.random.RandomState(seed)
    t0     = time.time()

    # ------------------------------------------------------------------
    # 3. Bootstrap loop
    # ------------------------------------------------------------------
    for b in range(1, n_loop + 1):
        try:
            # ---- 3.1 sample rows -------------------------------------
            idx = rng.choice(X.index, size=int(len(X) * frac), replace=True)
            X_b, y_b = X.loc[idx], y.loc[idx]

            # ---- 3.2 fit a fresh clone of the whole pipeline ---------
            pipe_b = clone(pipe)
            pipe_b.fit(X_b, y_b)

            # ---- 3.3 columns that reached the selector --------------
            surv_cols = pipe_b[:-1].get_feature_names_out()
            mask_local = pipe_b.named_steps[pipe_b.steps[-1][0]].get_support()
            selected_cols = surv_cols[mask_local]

            # ---- 3.4 map to master index and add vote ---------------
            votes += np.isin(master_cols, selected_cols)

        except Exception:
            print(f"[{b}/{n_loop}] ⚠️  exception – skipped "
                  f"(see stderr for details)", file=sys.stderr)
            traceback.print_exc()
            continue

        # ---- 3.5 heartbeat ------------------------------------------
        if b % echo_every == 0 or b == n_loop:
            elapsed = time.time() - t0
            print(f"[{time.strftime('%H:%M:%S')}] iteration {b}/{n_loop} "
                  f"(elapsed {elapsed:,.1f}s)")

    # ------------------------------------------------------------------
    # 4. Final frequency & keep mask
    # ------------------------------------------------------------------
    freq = votes / n_loop
    keep = freq >= cutoff
    return keep, freq, master_cols

def safe_log1p(X, eps: float = 1e-6):
    # --- convert to ndarray without copying when possible --------------
    if isinstance(X, (pd.Series, pd.DataFrame)):
        arr = X.to_numpy(dtype=float, copy=False)
    else:
        arr = np.asarray(X, dtype=float)

    # --- compute per-column shift -------------------------------------
    col_min = arr.min(axis=0)                    # 1-D vector
    shift   = np.where(col_min < -eps,           # only shift if <-eps
                       col_min - eps,
                       0.0)

    # --- broadcast & transform ----------------------------------------
    arr_shifted = arr - shift                   # row-wise broadcasting
    return np.log1p(arr_shifted)

log_branch = FunctionTransformer(safe_log1p, feature_names_out='one-to-one')


recip_branch = FunctionTransformer(lambda X: 1/(X + 1e-6),
                                   feature_names_out='one-to-one')

poly_branch  = PolynomialFeatures(degree=2,
                                  include_bias=False,
                                  interaction_only=True)

quant_branch = QuantileTransformer(output_distribution='uniform')

num_union = ColumnTransformer(
    transformers=[
        ('raw',   'passthrough', num_cols),       # a, b, …
        ('log',   log_branch,    num_cols),       # log(a), …
        ('recip', recip_branch,  num_cols),       # 1/a, …
        ('poly',  poly_branch,   num_cols),       # a*b, a*(1/b), …
        ('quant', quant_branch, num_cols)         # quantiles of a, b, …
    ],
    remainder='drop',
    sparse_threshold=0.3
)

full = ColumnTransformer(
    transformers=[
        ('num', num_union, num_cols),
        ('cat', 'passthrough', cat_cols)
    ],
    remainder='drop',
    sparse_threshold=0.3
)

preselect = Pipeline([
    ('prep',    full),
    ('var',     VarianceThreshold(1e-4)),
    ('corr',    CorrFilter(threshold=0.99, sample_size=10_000)),
    ('scale',   StandardScaler(with_mean=False)),
    ('sfrom',   SelectFromModel(
        estimator=ElasticNetCV(l1_ratio=[.1,.5,.9,1],
                               cv=5, max_iter=3000,
                               tol=1e-3, n_jobs=1),
        threshold='median'))
])


In [23]:
keep_mask, freq, master_cols = stability_mask(
    pipe       = preselect,
    X          = train.drop(columns=target),
    y          = train[target],
    n_loop     = 100,
    frac       = 0.10,
    cutoff     = 0.60,
    echo_every = 10
)


[16:31:24] iteration 10/100 (elapsed 403.9s)
[16:36:49] iteration 20/100 (elapsed 729.3s)
[16:41:46] iteration 30/100 (elapsed 1,025.7s)
[16:46:48] iteration 40/100 (elapsed 1,328.2s)
[16:52:00] iteration 50/100 (elapsed 1,639.7s)
[16:59:34] iteration 60/100 (elapsed 2,094.1s)
[17:05:15] iteration 70/100 (elapsed 2,435.0s)
[17:09:26] iteration 80/100 (elapsed 2,686.0s)
[17:13:32] iteration 90/100 (elapsed 2,931.8s)
[17:17:54] iteration 100/100 (elapsed 3,193.9s)


In [25]:
master_cols[keep_mask]

array(['num__raw__Age', 'num__raw__Height', 'num__raw__Weight',
       'num__raw__Duration', 'num__raw__Heart_Rate',
       'num__raw__Body_Temp', 'num__raw__BMI', 'num__raw__%HRMax',
       'num__raw__HRR', 'num__raw__TRIMP', 'num__raw__BMR',
       'num__raw__Keytel', 'num__log__Age', 'num__log__Duration',
       'num__log__HRR', 'num__log__TRIMP', 'num__log__Keytel',
       'num__recip__Age', 'num__recip__Duration', 'num__recip__%HRMax',
       'num__recip__Thermal_Load', 'num__recip__Keytel',
       'num__poly__Age Height', 'num__poly__Age Weight',
       'num__poly__Age Duration', 'num__poly__Age Heart_Rate',
       'num__poly__Age BMI', 'num__poly__Age HRR',
       'num__poly__Age Keytel', 'num__poly__Height Duration',
       'num__poly__Height Heart_Rate', 'num__poly__Height Body_Temp',
       'num__poly__Height BMI', 'num__poly__Height HRMax',
       'num__poly__Height %HRMax', 'num__poly__Height HRR',
       'num__poly__Height TRIMP', 'num__poly__Weight Duration',
       'num_

In [26]:
keep_mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [28]:
len(master_cols)

78

In [29]:
len(master_cols[keep_mask])

78